In [ ]:
__author__ = 'Alice Jacques <alice.jacques@noirlab.edu>, NOIRLab Astro Data Lab Team <datalab@noirlab.edu>' 
__version__ = '20211112' #yyyymmdd 
__datasets__ = ['ls_dr9','sdss_dr16','gaia_dr2','des_dr1','smash_dr2','unwise_dr1','allwise','nsc_dr2'] 
__keywords__ = ['crossmatch','joint query','mydb','vospace','image cutout']

# How to use the pre-crossmatched tables at Astro Data Lab

by Alice Jacques and the NOIRLab Astro Data Lab Team

### Table of contents
* [Goals & notebook summary](#goals)
* [Disclaimer & attribution](#attribution)
* [Imports & setup](#import)
* [Authentication](#auth)
* [Accessing the pre-crossmatched tables](#access)
* [Writing a JOIN query](#joinquery)
* [Joint query of LS and SDSS catalogs](#lssdss)
* [Crossmatch a user-provided table and a pre-crossmatched table](#usertable)
* [Speed test](#speedtest)
* [Appendix](#appendix)
* [Resources & references](#refs)

<a class="anchor" id="goals"></a>
# Goals

* Learn how to use a pre-crossmatched table to do a joint query on two Data Lab data sets
* Learn how to do an efficient crossmatch of a user-provided data table against a Data Lab pre-crossmatched table

# Summary

#### Crossmatch table naming template
The crossmatch tables at Astro Data Lab are named as follows:

`schema1.xNpN__table1__schema2__table2`

where the N in NpN encode the numerical value of the crossmatching radius (since dots '.' are not allowed in table names).

Example: 

`ls_dr9.x1p5__tractor__nsc_dr2__object`

is a crossmatch table (indicated by the leading x after the dot '.'), located in the ls_dr9 schema, and it crossmatches the **ls_dr9.tractor** table with the **nsc_dr2.object** table (which lives in the nsc_dr2 schema) within a 1.5 arcseconds radius ('1p5') .

This is admittedly long, but clean, consistent, and most importantly, parsable. The use of double-underscores '\__' is to distinguish from single underscores often used in schema and table names.


#### Columns in crossmatch tables
All crossmatch tables shall be minimalist, i.e. have only these columns: id1,ra1,dec1,id2,ra2,dec2,distance.
Column descriptions in the crossmatch table shall contain the original column names in parentheses (makes it parsable).

For example:

`ls_dr9.x1p5__tractor__nsc_dr2__object`

| Column   | Description                                     | Datatype |
|----------|-------------------------------------------------|----------|
| id1      | ID in left/first table (ls_id)                  | BIGINT   |
| ra1      | Right ascension in left/first table (ra)        | DOUBLE   |
| dec1     | Declination in left/first table (dec)           | DOUBLE   |
| id2      | ID in right/second table (id)                   | VARCHAR  |
| ra2      | Right ascension in right/second table (ra)      | DOUBLE   |
| dec2     | Declination in right/second table (dec)         | DOUBLE   |
| distance | Distance between ra1,dec1 and ra2,dec2 (arcsec) | DOUBLE   |


#### Datatypes in crossmatch tables

* The column data types in a crossmatch table for columns id1 and id2 shall be retained from the mother tables. The example above, BIGINT, is valid in many cases, but need not be for all data sets.
* The data types for columns ra1, dec1, ra2, dec2 shall be DOUBLE, which they usually will be anyway.
* The column distance can be either REAL or DOUBLE.

#### Overview

* **The following 5 data sets are considered the main reference tables** and are crossmatched against all data sets (if there is sky overlap) and when a new data set is ingested:  
    - latest gaia_drN.gaia_source    
    - latest nsc_drN.object
    - latest unwise_drN.object
    - allwise.source  
    - latest sdss_drN.specobj
* **"Crossmatch" means for now "single nearest neighbor"** (and this is the current mode at Data Lab).
* **Object tables only**, not single epoch measurements or metadata tables.
* For every crossmatch table with table1 as the left/first table and table2 as the right/second table, there exists a corresponding crossmatch table with table2 as the left/first table and table1 as the right/second table. 
    - For example, `allwise.x1p5__source__des_dr2__main` and `des_dr2.x1p5__main__allwise__source`.


The list of available crossmatch tables can be viewed on our query interface [here](https://datalab.noirlab.edu/query.php) under their respective schema.

<a class="anchor" id="attribution"></a>
# Disclaimer & attribution
If you use this notebook for your published science, please acknowledge the following:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, http://dx.doi.org/10.1117/12.2057445

* Data Lab disclaimer: https://datalab.noirlab.edu/disclaimers.php

<a class="anchor" id="import"></a>
# Imports and setup

In [ ]:
# std lib
from getpass import getpass

# 3rd party
from astropy.utils.data import download_file  #import file from URL
from matplotlib.ticker import NullFormatter
import pylab as plt
import matplotlib
%matplotlib inline

# Data Lab
from dl import authClient as ac, queryClient as qc, storeClient as sc
from dl.helpers.utils import convert # converts table to Pandas dataframe object

<a class="anchor" id="auth"></a>
# Authentication
Much of the functionality of Data Lab can be accessed without explicitly logging in (the service then uses an anonymous login). But some capacities, for instance saving the results of your queries to your virtual storage space, require a login (i.e. you will need a registered user account).

If you need to log in to Data Lab, issue this command, and respond according to the instructions:

In [ ]:
#ac.login(input("Enter user name: (+ENTER) "),getpass("Enter password: (+ENTER) "))
ac.whoAmI()

<a class="anchor" id="access"></a>
# Accessing the pre-crossmatched tables 
We can use Data Lab's Query Client to access the pre-crossmatched tables hosted by Data Lab. First let's get a total count of the number of objects (nrows) in SDSS DR16 that are also in LS DR9:

In [ ]:
%%time
query="SELECT nrows FROM tbl_stat WHERE schema='sdss_dr16' and tbl_name='x1p5__specobj__ls_dr9__tractor'"

# Call query manager
response = qc.query(sql=query)
print(response)

Now let's print just the first 100 rows:

In [ ]:
%%time
query = "SELECT * FROM sdss_dr16.x1p5__specobj__ls_dr9__tractor LIMIT 100"
result = qc.query(sql=query,fmt='pandas')
result

<a class="anchor" id="joinquery"></a>
# Writing a JOIN query
In order to extract only the relevant columns pertaining to our science question from multiple data tables, we may write a query that uses a JOIN statement. There are 4 main types of JOIN statements that we could use, and which one we decide to choose depends on how we want the information to be extracted. 
1. **(INNER) JOIN**: Returns rows that have matching values in both tables
2. **LEFT (OUTER) JOIN**: Returns all rows from the left table, and the matched rows from the right table
3. **RIGHT (OUTER) JOIN**: Returns all rows from the right table, and the matched rows from the left table
4. **FULL (OUTER) JOIN**: Returns all rows when there is a match in either left or right table

Take a moment to look over the figure below outlining the various JOIN statement types.  
NOTE: the default JOIN is an `INNER JOIN`.

<img src='join.png'></img>

### `JOIN LATERAL`

In nearest neighbor crossmatch queries, we use `JOIN LATERAL`, which is like a SQL foreach loop that will iterate over each row in a result set and evaluate a subquery using that row as a parameter.

<a class="anchor" id="lssdss"></a>
# Joint query of LS and SDSS catalogs
Here we will examine spectroscopic redshifts from SDSS DR16 and photometry from LS DR9. The two crossmatch tables related to these two catalogs are: 
`ls_dr9.x1p5__tractor__sdss_dr16__specobj` and `sdss_dr16.x1p5__specobj__ls_dr9__tractor`. The choice of which of these two crossmatch tables to use should be based on the science question being posed. For instance, the question *'how does a galaxy's structure change with redshift?'* is dependent on the redshift values obtained from SDSS DR16, so we should use the crossmatch table that has SDSS DR16 as the first table. So, the relevant information we want from our 3 tables of interest for this example are:

1. "X" = `sdss_dr16.x1p5__specobj__ls_dr9__tractor`
    - **ra1** (RA of sdss object)
    - **dec1** (Dec of sdss object)
2. "S" = `sdss_dr16.specobj`
    - **z** (redshift)
3. "L" = `ls_dr9.tractor`
    - **ra** (RA of ls object)
    - **dec** (Dec of ls object)
    - **g_r** (computed g-r color)

### Write the query
Now that we know what we want and where we want it from, let's write the query and then print the results on screen. Here we use two join statements: the first will search in the SDSS DR16 `specobj` table for rows that have the same SDSS id value (`specobjid`) as in the pre-crossmatched table (`id1`) and retrieve the desired columns from the SDSS DR16 `specobj` table. The second will search in the LS DR9 `tractor` table for rows that have the same LS id value (`ls_id`) as in the pre-crossmatched table (`id2`) and retrieve the desired columns from the LS DR9 `tractor` table.

In [ ]:
query = ("""SELECT 
           X.ra1 as ra_sdss, X.dec1 as dec_sdss, S.z,
           L.ra as ra_ls, L.dec as dec_ls, L.g_r
         FROM sdss_dr16.x1p5__specobj__ls_dr9__tractor as X 
         JOIN sdss_dr16.specobj as S ON X.id1 = S.specobjid 
         JOIN ls_dr9.tractor AS L ON X.id2 = L.ls_id
         WHERE X.ra1 BETWEEN %s and %s and X.dec1 BETWEEN %s and %s
         LIMIT 10000
         """) %(110,200,7.,40.)  #large region
print(query)

In [ ]:
%%time
df = qc.query(sql=query,fmt='pandas')
df

### Saving results to VOSpace
VOSpace is a convenient storage space for users to save their work. It can store any data or file type. We can save the results from the same query to our virtual storage space:

In [ ]:
%%time
response = qc.query(sql=query,fmt='csv',out='vos://testresult.csv')

Let's ensure the file was saved in VOSpace:

In [ ]:
sc.ls(name='vos://testresult.csv')

Now let's remove the file we just saved to VOSpace:

In [ ]:
sc.rm (name='vos://testresult.csv')

Let's ensure the file was removed from VOSpace:

In [ ]:
sc.rm (name='vos://testresult.csv')

### Saving results to MyDB
MyDB is a useful OS remote per-user relational database that can store data tables. Furthermore, the results of queries can be directly saved to MyDB, as we show in the following example:

In [ ]:
%%time
response = qc.query(sql=query, fmt='csv', out='mydb://testresult')

Ensure the table has been saved to MyDB by calling the `mydb_list()` function, which will list all tables currently in a user's MyDB:

In [ ]:
print(qc.mydb_list(),"\n")

Now let's drop the table from our MyDB.

In [ ]:
qc.mydb_drop('testresult')

Ensure it has been removed by calling the `mydb_list()` function again:

In [ ]:
print(qc.mydb_list(),"\n")

<a class="anchor" id="usertable"></a>
# Crossmatch a user-provided data table and a pre-crossmatched table
We can construct a query to run a crossmatch in the database using the `q3c_join()` function, which identifies all matching objects within a specified radius in degrees (see details on [using Q3C functions](https://github.com/segasai/q3c)). For this example, we will search only for the single nearest neighbor. For different examples of crossmatching, see our [How to crossmatch tables notebook](https://github.com/noaodatalab/notebooks-latest/blob/master/04_HowTos/CrossmatchTables/How_to_crossmatch_tables.ipynb). 

First, let's query a small selection of sample data from the Data Lab database and store it in MyDB as `gaia_sample`. This will act as our "user-provided table".

In [ ]:
query = """SELECT source_id, ra, dec, parallax, pmra, pmdec 
            FROM gaia_dr2.gaia_source 
            WHERE ra<200 AND ra>124 AND random_id<10 
            LIMIT 10000"""
print(query)

In [ ]:
%%time
response = qc.query(sql=query,out='mydb://gaia_sample',drop=True)

### Write a crossmatch query

Next let's crossmatch our `gaia_sample` table with Data Lab's pre-crossmatched table between SMASH DR2 and allWISE `smash_dr2.x1p5__object__allwise__source`. We'll write our crossmatch query using the `q3c_join()` function as well as the `q3c_dist()` function, searching for the nearest neighbor within a 1.5 arcsec radius (which must be converted into degrees for the query, so we divide by 3600.0). We will then save it in MyDB as `gaia_sample_xmatch`.

In [ ]:
%%time
qu = """SELECT
        G.source_id, ss.id1, ss.id2, G.ra, G.dec, ss.ra1, ss.dec1, ss.ra2, ss.dec2,
        (q3c_dist(G.ra, G.dec, ss.ra1, ss.dec1)*3600.0) as dist_arcsec
        FROM mydb://gaia_sample AS G
        JOIN LATERAL (
            SELECT S.id1, S.id2, S.ra1, S.dec1, S.ra2, S.dec2
            FROM 
                smash_dr2.x1p5__object__allwise__source AS S
            WHERE 
                q3c_join(G.ra, G.dec, S.ra1, S.dec1, 1.5/3600.0)
            ORDER BY
                q3c_dist(G.ra, G.dec, S.ra1, S.dec1)
            ASC LIMIT 1
            ) AS ss ON true
    """
resp = qc.query(sql=qu,out='mydb://gaia_sample_xmatch',drop=True)

We can query the newly created table from MyDB and convert it into a Pandas Dataframe object in order to print it on screen:

In [ ]:
query = "SELECT * FROM mydb://gaia_sample_xmatch"
df = qc.query(sql=query,fmt='pandas')
df

### Write the joint query
Now we can write a query using the JOIN statement in order to extract the columns we want from our tables of interest. Just as in the previous section, let's first make an outline of which tables we'd like to extract columns from.

1. "X" = `mydb://gaia_sample_xmatch`
    - **source_id** (source id from gaia dr2)
    - **id1** (source id from smash dr1) 
    - **id2** (source id from allwise)
    - **ra** (RA value from gaia dr2)
    - **dec** (Dec value from gaia dr2)
2. "s" = `smash_dr2.object`
    - **gmag** (weighted-avarage, calibrated g-band magnitude, 99.99 if no detection)
3. "a" = `allwise.source` 
    - **w1mpro** (W1 magnitude measured with profile-fitting photometry)
4. "g" = `mydb://gaia_sample`
    - **parallax**
    - **pmra** (proper motion in right ascension direction)
    - **pmdec** (proper motion in declination direction)
    
    

In [ ]:
query = ("""SELECT 
           X.source_id, X.id1, X.id2, X.ra, X.dec,
           s.gmag, a.w1mpro, g.parallax, g.pmra, g.pmdec
         FROM mydb://gaia_sample_xmatch as X 
         JOIN smash_dr2.object as s ON X.id1 = s.id 
         JOIN allwise.source AS a ON X.id2 = a.source_id
         JOIN mydb://gaia_sample AS g ON X.source_id = g.source_id
         """)
print(query)

In [ ]:
%%time
df = qc.query(sql=query,fmt='pandas')
df

<a class="anchor" id="speedtest"></a>
# Speed test
Here we compare the speed of using the `q3c_join()` function to crossmatch directly in a JOIN query (`query1`) versus using a pre-crossmatched table (`query2`). We retrieve the same specified columns for the two queries. We will see that `query2` retrieves 1000 results much faster than `query1` can retrieve 1000 results.

#### First, running the crossmatch ourselves:

In [ ]:
%%time
query1 = """SELECT
           a.source_id as id1, a.ra as ra1, a.dec as dec1,
           gg.specobjid as id2, gg.ra as ra2, gg.dec as dec2,
           (q3c_dist(a.ra, a.dec, gg.ra, gg.dec)*3600) as distance 
         FROM 
            allwise.source AS a
         INNER JOIN LATERAL (
            SELECT s.specobjid, s.ra, s.dec
            FROM 
                sdss_dr16.specobj AS s
            WHERE
                q3c_join(a.ra, a.dec, s.ra, s.dec, 1.5/3600.0)
            ORDER BY
                q3c_dist(a.ra, a.dec, s.ra, s.dec)
            ASC LIMIT 1
        ) as gg ON true
        LIMIT 1000
"""
df1 = qc.query(sql=query1,fmt='pandas')
df1 = df1.sort_values('id1')

#### Now, the same but using pre-crossmatched tables:

In [ ]:
%%time
query2 = """SELECT 
           X.id1, X.id2, X.ra1, X.dec1, X.ra2, X.dec2, X.distance
         FROM 
             allwise.x1p5__source__sdss_dr16__specobj as X 
         LIMIT 1000
"""
df2 = qc.query(sql=query2,fmt='pandas')
df2 = df2.sort_values('id1')

### For completeness, we switch the order of the queries and query from a different catalog.
We again select objects from two catalogs and retrieve the same specified columns for two queries. `query3` uses a pre-crossmatched table in a JOIN query and `query4` crossmatches directly in the JOIN query. We will see that `query3` retrieves 1000 results much faster than `query4` can retrieve 1000 results.

#### First, using pre-crossmatched tables:

In [ ]:
%%time
query3 = """SELECT 
           X.id1, X.id2, X.ra1, X.dec1, X.ra2, X.dec2, X.distance
         FROM 
             unwise_dr1.x1p5__object__sdss_dr16__specobj as X 
         LIMIT 1000
         """
df3 = qc.query(sql=query3,fmt='pandas')
df3 = df3.sort_values('id1')

#### Now, running the crossmatch ourselves: 

In [ ]:
%%time
query4 = """SELECT
           u.unwise_objid as id1, u.ra as ra1 ,u.dec as dec1,
           ss.specobjid as id2, ss.ra as ra2, ss.dec as dec2,
           (q3c_dist(u.ra, u.dec, ss.ra, ss.dec)*3600.0) as distance 
         FROM 
            unwise_dr1.object AS u
         INNER JOIN LATERAL (
            SELECT s.specobjid, s.ra, s.dec
            FROM 
                sdss_dr16.specobj AS s
            WHERE
                q3c_join(u.ra, u.dec, s.ra, s.dec, 1.5/3600.0)
            ORDER BY
                q3c_dist(u.ra, u.dec, s.ra, s.dec)
            ASC LIMIT 1
        ) as ss ON true
        LIMIT 1000
"""
df4 = qc.query(sql=query4,fmt='pandas')
df4 = df4.sort_values('id1')

<a class="anchor" id="appendix"></a>
# Appendix

A clear benefit of pre-crossmatched tables is that they contain the positions of the same objects in two datasets. We can use this to e.g. fetch images of an object from both surveys.

## A1. unWISE DR1 vs LS DR9 
Here we will compare two images of the same object from two different catalogs, unWISE DR1 and LS DR9. 

### Function to retrieve cutouts

In [ ]:
def make_cutout_comparison_table(ra_in1, dec_in1, layer1, layer2, pixscale, ra_in2=None, dec_in2=None):
    """
    Obtain color JPEG images from Legacy Survey team cutout tool at NERSC
    """    
    img1 = []
    img2 = []
    
    for i in range(len(ra_in1)):
        cutout_url1 = "https://www.legacysurvey.org/viewer/cutout.jpg?ra=%g&dec=%g&layer=%s&pixscale=%s" % (ra_in1[i],dec_in1[i],layer1,pixscale)
        img = plt.imread(download_file(cutout_url1,cache=True,show_progress=False,timeout=120))
        img1.append(img)
        
        cutout_url2 = "https://www.legacysurvey.org/viewer/cutout.jpg?ra=%g&dec=%g&layer=%s&pixscale=%s" % (ra_in2[i],dec_in2[i],layer2,pixscale)
        img = plt.imread(download_file(cutout_url2,cache=True,show_progress=False,timeout=120))
        img2.append(img)

    return img1,img2

### Function to generate plots

In [ ]:
def plot_cutouts(img1,img2,cat1,cat2):
    """
    Plot images in two rows with 5 images in each row
    """
    fig = plt.figure(figsize=(21,7))

    for i in range(len(img1)):
        ax = fig.add_subplot(2,6,i+1)
        ax.imshow(img1[i])
        ax.xaxis.set_major_formatter(NullFormatter())
        ax.yaxis.set_major_formatter(NullFormatter())
        ax.tick_params(axis='both',which='both',length=0)
        ax.text(0.02,0.93,'ra=%.5f'%list_ra1[i],transform=ax.transAxes,fontsize=12,color='white')
        ax.text(0.02,0.85,'dec=%.5f'%list_dec1[i],transform=ax.transAxes,fontsize=12,color='white')
        ax.text(0.02,0.77,cat1,transform=ax.transAxes,fontsize=12,color='white')

        ax = fig.add_subplot(2,6,i+7)
        ax.imshow(img2[i])
        ax.xaxis.set_major_formatter(NullFormatter())
        ax.yaxis.set_major_formatter(NullFormatter())
        ax.tick_params(axis='both',which='both',length=0)
        ax.text(0.02,0.93,'ra=%.5f'%list_ra2[i],transform=ax.transAxes,fontsize=12,color='white')
        ax.text(0.02,0.85,'dec=%.5f'%list_dec2[i],transform=ax.transAxes,fontsize=12,color='white')
        ax.text(0.02,0.77,cat2,transform=ax.transAxes,fontsize=12,color='white')

    plt.subplots_adjust(wspace=0.02, hspace=0.03)

###  Write query to randomly select five targets (RA/Dec positions) from unWISE DR1 and LS DR9 crossmatch table
... then save them as arrays and set the captions, layers, and pixscale. Finally we plot the cutout images.

In [ ]:
%%time
q = """SELECT ra1,dec1,ra2,dec2 
        FROM unwise_dr1.x1p5__object__ls_dr9__tractor 
        WHERE ra1>300 AND dec1>33 
        ORDER BY random() 
        LIMIT 5"""

r = qc.query(sql=q,fmt='pandas')

list_ra1=r['ra1'].values       # ".values" convert to numpy array
list_dec1=r['dec1'].values
list_ra2=r['ra2'].values       
list_dec2=r['dec2'].values

cat1='unWISE'
cat2='ls dr9'
layer1='unwise-neo6'
layer2='ls-dr9'
pixscale='0.3'
img1,img2 = make_cutout_comparison_table(list_ra1,list_dec1,layer1,layer2,
                                         pixscale,list_ra2,list_dec2)
plot_cutouts(img1,img2,cat1,cat2)

## A2. SDSS vs DES DR1
Here we will compare two images of the same object from two different catalogs, SDSS and DES DR1.

### Write query to randomly select five targets (RA/Dec positions) from SDSS DR16 and DES DR1 crossmatch table
... then save them as arrays and set the captions, layers, and pixscale. Finally we plot the cutout images.

In [ ]:
%%time
q = """SELECT ra1,dec1,ra2,dec2 
        FROM sdss_dr16.x1p5__specobj__des_dr1__main 
        ORDER BY random() 
        LIMIT 5"""

r = qc.query(sql=q,fmt='pandas')

list_ra1=r['ra1'].values       # ".values" convert to numpy array
list_dec1=r['dec1'].values
list_ra2=r['ra2'].values       
list_dec2=r['dec2'].values

cat1='sdss dr16'
cat2='des dr1'
layer1='sdss'
layer2='des-dr1'
pixscale='0.25'
img1,img2 = make_cutout_comparison_table(list_ra1,list_dec1,layer1,layer2,
                                         pixscale,list_ra2,list_dec2)
plot_cutouts(img1,img2,cat1,cat2)

## A3. Cool galaxy finds: SDSS vs DES DR1

We compare two images of the same galaxy from two different catalogs, SDSS and DES DR1. We use a list of identified galaxies (RA/Dec positions) to compare the difference in observable features and quality between the two catalogs.

First we import the CSV file of identified galaxies (RA/Dec positions) into MyDB:

In [ ]:
qc.mydb_import('gals','./gals.csv',drop=True)

We write the query to select the first five RA/Dec positions from our table. We then save them as arrays and set the captions, layers, and pixscale. Finally we plot the cutout images.

In [ ]:
%%time
qg = """SELECT ra,dec 
        FROM mydb://gals 
        LIMIT 5"""
rg = qc.query(sql=qg)
rp = convert(rg)
list_ra1=rp['ra'].values 
list_dec1=rp['dec'].values
list_ra2=rp['ra'].values
list_dec2=rp['dec'].values

cat1='sdss dr16'
cat2='des dr1'
layer1='sdss'
layer2='des-dr1'
pixscale='0.5'

img1,img2 = make_cutout_comparison_table(list_ra1,list_dec1,layer1,layer2,
                                        pixscale,ra_in2=list_ra1,dec_in2=list_dec1)
plot_cutouts(img1,img2,cat1,cat2)

We write the next query to select the next five RA/Dec positions from our table. We then save them as arrays and set the captions, layers, and pixscale. Finally we plot the cutout images.

In [ ]:
%%time
qg = """SELECT ra,dec 
        FROM mydb://gals 
        LIMIT 5 
        OFFSET 5"""
rg = qc.query(sql=qg)
rp = convert(rg)
list_ra1=rp['ra'].values      
list_dec1=rp['dec'].values
list_ra2=rp['ra'].values     
list_dec2=rp['dec'].values

img1,img2 = make_cutout_comparison_table(list_ra1,list_dec1,layer1,layer2,
                                        pixscale,ra_in2=list_ra1,dec_in2=list_dec1)
plot_cutouts(img1,img2,cat1,cat2)

We write the next query to select the last five RA/Dec positions from our table. We then save them as arrays and set the captions, layers, and pixscale. Finally we plot the cutout images.

In [ ]:
%%time
qg = """SELECT ra,dec 
        FROM mydb://gals 
        LIMIT 5 
        OFFSET 10"""
rg = qc.query(sql=qg)
rp = convert(rg)
list_ra1=rp['ra'].values    
list_dec1=rp['dec'].values
list_ra2=rp['ra'].values     
list_dec2=rp['dec'].values

img1,img2 = make_cutout_comparison_table(list_ra1,list_dec1,layer1,layer2,
                                        pixscale,ra_in2=list_ra1,dec_in2=list_dec1)
plot_cutouts(img1,img2,cat1,cat2)

<a class="anchor" id="refs"></a>
# Resources & references

W3Schools: SQL Joins https://www.w3schools.com/sql/sql_join.asp  
Legacy Survey Sky Browser: https://www.legacysurvey.org/viewer#NGC%203098